In [ ]:
import multiprocessing
from multiprocessing import Process, Queue

import time
from lxml import etree
import requests
from bs4 import BeautifulSoup

#url = 'https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1'
url = 'https://movies.yahoo.com.tw/movie_thisweek.html'
infos = []
def check_req_url(url): #測試請求網址是否請求成功
    r = requests.get(url) #請求網址
    #print(resp.status_code) #錯誤時404,成功時200
    if r.status_code != 200:  #如果請求失敗
        print('Invalid url:', r.url) #印出請求失敗的網址
        return "fail" #回傳失敗提示訊息
    else:
        return r.text #回傳請求成功的html文字

def get_week_new_movies(webpage): #抓取電影資訊
  soup = BeautifulSoup(webpage) #網頁解析
  for d in soup.find_all('div', class_="release_info"):
      info = {}
      info['中文名稱'] = d.find('div', class_='release_movie_name').find('a', class_='gabtn').text.strip()
      info['英文名稱'] = d.find('div', class_='release_movie_name').find('div', class_='en').a.text.strip()
      info['上映日期'] = d.find('div', class_='release_movie_time').text.split('上映日期 ： ')[1]
      info['期待度'] = ''
      info['劇情介紹'] = d.find('div', class_='release_text').find('span', class_='jq_text_overflow_180 jq_text_overflow_href_list').text.strip()
      info['連接介紹'] = d.find('div', class_='release_movie_name').find('a', class_='gabtn').get('href')
      net = d.find('div', class_='release_movie_name').find('a', class_='gabtn').get('href')
      page = check_req_url(net)
      if page:
        subsoup = BeautifulSoup(page)   
        level_name = subsoup.find('div', class_='level_name_box').find_all('a',class_='gabtn')
        str =''
        for level in level_name:
          str += level.text.strip()+','
        #print(str)
        info['Label'] = str   
      else:
        print("fail")

      if d.find('div', class_='leveltext'):    
          info['期待度'] = d.find('div', class_='leveltext').span.text
      infos.append(info) 
  return infos

if __name__ == '__main__':
  href = []
  for i in range(1,20):
    url = 'https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=%d'%(i)
    webpage = check_req_url(url)
    if webpage:
      infos = get_week_new_movies(webpage)
      for info in infos:
        print(info)

    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    for d in soup.find_all(class_="nexttxt"):
      ll = d.a.get('href')
    href.append(ll)
    for j in href: 
      print("j = ")
      print(j)
      print("\n")
      webpage = check_req_url(j)
      if webpage:
        infos = get_week_new_movies(webpage)
      r = requests.get(j)
      soup = BeautifulSoup(r.text)
      for d in soup.find_all(class_="nexttxt"):
        ll = d.a.get('href')
      href.append(ll)
  for info in infos:
    print(info)




KeyboardInterrupt: ignored

In [ ]:
import multiprocessing
from multiprocessing import Process, Queue

import time
from lxml import etree
import requests
from bs4 import BeautifulSoup

#url = 'https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1'
url = 'https://movies.yahoo.com.tw/movie_thisweek.html'
infos = []
def check_req_url(url): #測試請求網址是否請求成功
    r = requests.get(url) #請求網址
    #print(resp.status_code) #錯誤時404,成功時200
    if r.status_code != 200:  #如果請求失敗
        print('Invalid url:', r.url) #印出請求失敗的網址
        return "fail" #回傳失敗提示訊息
    else:
        return r.text #回傳請求成功的html文字

def get_week_new_movies(webpage): #抓取電影資訊
  soup = BeautifulSoup(webpage) #網頁解析
  exist = soup.find_all('div', id = 'content_l')
  if exist:
    for d in soup.find_all('div', id="content_l"):
        info = {}
        info['中文名稱'] = d.find('div', class_='movie_intro_info_r').find('h1').text.strip()
        info['英文名稱'] = d.find('div', class_='movie_intro_info_r').find('h3').text.strip()
        info['上映日期'] = d.find('span').text.strip()
        info['劇情介紹'] = d.find('div', class_='gray_infobox storeinfo').find('div', class_='gray_infobox_inner').find('span',id='story').text.strip()
        # info['連接介紹'] = d.find('div', class_='release_movie_name').find('a', class_='gabtn').get('href')
        # net = d.find('div', class_='release_movie_name').find('a', class_='gabtn').get('href')
        level_name = d.find('div', class_='movie_intro_info_r').find_all('div',class_='level_name')
        str =''
        for level in level_name:
          str += level.a.text.strip()+',' 
        info['Label'] = str   
        infos.append(info) 
    return infos
  else:
    return 0

if __name__ == '__main__':
  headers = {
      'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36'
  }
# url = 'https://movies.yahoo.com.tw/movieinfo_main/9'
# check_req_url(url)
  for i in range(1,10000):
    print(i)
    url = 'https://movies.yahoo.com.tw/movieinfo_main/%d'%(i)+''
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text) #網頁解析
    #exist = soup.find_all('div', id = 'content_l')
   # print(soup.find_all('div', class_ = 'movie_intro_info_r').find('h1').text.strip())
    if soup.find_all('div', class_='movie_intro_info_r'):
      infos = get_week_new_movies(r.text)
    else:
      print("NON")
      continue
  import pandas as pd

  df = pd.DataFrame(infos)
  df.to_csv('movies.csv', encoding='utf_8_sig')     
      
  df    
    

  # url = 'https://movies.yahoo.com.tw/movieinfo_main/1'
  # r = requests.get(url, headers=headers)
  # infos = get_week_new_movies(r.text)

  # webpage = check_req_url(url)
  # if webpage:
  #   infos = get_week_new_movies(webpage)
  # for info in infos:
  #   print(info)


1
2
3
4
5
6
7
8
9
NON
10
11
12
13


In [ ]:
import multiprocessing
from multiprocessing import Process, Queue

import time
from lxml import etree
import requests
from bs4 import BeautifulSoup

#url = 'https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1'
#url = 'https://movies.yahoo.com.tw/movie_thisweek.html'
infos = []
url = 'https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=1'


href = ['https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=1'];
# for i in range(1,3):
#   url = 'https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=%d'%(i)+''
#   r = requests.get(url)
#   soup = BeautifulSoup(r.text)
#   for d in soup.find_all(class_="nexttxt"):
#     ll = d.a.get('href')
#   print(ll)

for i in range(1,10):
  url = 'https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=%d'%(i)
  
  for j in href: 
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    for d in soup.find_all(class_="nexttxt"):
      ll = d.a.get('href')
    href.append(ll)
  print(href)
  

  #   url = 'https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=%d'%(i)+''
  #   r = requests.get(url)
  #   soup = BeautifulSoup(r.text)
  #   for d in soup.find_all(class_="nexttxt"):
  #     ll = d.a.get('href')
  #   href.append(ll)
  # print(href) 
    


['http://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=2', 'http://movies.yahoo.com.tw/moviegenre_result.html?genre_id=2&page=2']


In [ ]:
set1 = set()# 建立空的集合

set2 = {1, 2, 3, 4, 5}

# 從串列 (List) 來建立集合
set3 = set([i for i in range(20, 30)])

# 從數組 (Tuple) 來建立集合
set4 = set((10, 20, 30, 40, 50))

# 集合不會包含重複的資料, 你可以從 set5 印出來的結果得知
set5 = set((1, 1, 1, 2, 2, 3))
ls = [1,2,1,3,2,1,2,3,1,3,2,1,2,3,]
aa = set(ls)
print(aa)
print(set1)
print(set2)
print(set3)
print(set4)
print(set5)

{1, 2, 3}
set()
{1, 2, 3, 4, 5}
{20, 21, 22, 23, 24, 25, 26, 27, 28, 29}
{40, 10, 50, 20, 30}
{1, 2, 3}


In [ ]:
from bs4 import BeautifulSoup
import requests
 
for page in range(1, 4):  # 執行1~3頁
 
    # 連結網站
    response = requests.get(
        "https://www.inside.com.tw/tag/AI?page=" + str(page))
 
    # HTML原始碼解析
    soup = BeautifulSoup(response.text, "html.parser")
 
    # 取得所有class為post_title的<h3>
    titles = soup.find_all("h3", {"class": "post_title"})
 
    print(f"====================第{str(page)}頁====================")
    for title in titles:
        print(title.select_one("a").getText())  # 取得連結的標題文字

====================第1頁====================
iKala 十週年記者會：疫情衝擊擴張版圖 年營收預計突破十億元台幣
【Howie 商業投資】追上 Google！ 臉書推出 Droidlet 加速智慧機器人開發
【開放文化觀點】難用、歧視又卸責！美國政府的人臉辨識服務如何辜負成千上萬失業者？
【塞掐 Side Chat】EP 23｜進了人人稱羨的超大企業，為何離職創業？ft. AI 新創 A Team
新創助傳統產業變智慧！SparkLabs Taipei DemoDay 5 聚焦數位轉型
橫跨軟硬體與解決方案，研華董事長劉克振親揭 2025 AIoT 佈局
Appier Q2 財報：營收 28 億日圓 YoY 增 50%，並於 6 月實現首次營業獲利
科技部 AI 教練助攻！東奧舉重郭婞淳奪金背後的視覺分析工具
IN 啦！「鷹眼系統」到底是什麼科技？
【電車世代】安永 AI 推測：全球主要市場電動車銷量將在 2033 超越油車
高通攜手富士康推出基於 Qualcomm Cloud AI 100 的 Gloria AI Edge Box 邊際 AI 設備，獲越南 BKAV 採納並用於智慧邊際
Computex 2021 ： Intel 強調 AI 以 XPU 架構為核心戰略，以 Xeon 為核心搭配異構運算解決複雜而廣泛的運算問題
高通在 Snapdragon 888 展示基於軟體的 AI 影片解碼器，可達成 1,280 x 704 30fps 的即時解碼
NVIDIA 遊戲夥伴夏日迎接 RTX 光追與 DLSS 加持，以樂高 拼砌旅程、毀滅戰士 永恆為首的新遊戲將獲得絕佳光影效果
全球首創認知智慧超高畫質顯示器 Sony BRAVIA XR 系列仿人腦高效分析運作呈現極致影像
NVIDIA 針對工業應用推出工規級 Jetson AGX Xavier 工業模組，具備寬溫耐性與抗衝擊與振動特性
韓國AI女團Eternity讓你怕怕的嗎？如何避開「恐怖谷現象」，降低不舒服感受？
Google 採用 AI 技術設計新一代 TPU ，僅需 6 小時就達到人類需要花費數個月的成果
Computex 2021 ： NVIDIA 邀請台灣智慧駕駛、勤崴國際分享台灣當前自動駕駛的產業現況
====================第2頁====